In [18]:
%pip uninstall lightgbm

In [1]:
pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"

DEPRECATION: --install-option is deprecated because it forces pip to use the 'setup.py install' command which is itself deprecated. pip 23.1 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11358
DEPRECATION: --no-binary currently disables reading from the cache of locally built wheels. In the future --no-binary will not influence the wheel cache. pip 23.1 will enforce this behaviour change. A possible replacement is to use the --no-cache-dir option. You can use the flag --use-feature=no-binary-enable-wheel-cache to test the upcoming behaviour. Discussion can be found at https://github.com/pypa/pip/issues/11453
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached lightgbm-3.3.5.tar.gz (1.5 MB)
  Preparing metadata (setup.py) ... done
  DEPRECATION: lightgbm is being installed using the legacy 'setup.py install' method, because

In [1]:
# import sys
# #!{sys.executable} -m pip install catboost
# %pip install catboost
# %pip install xgboost
# %pip install lightgbm
# %pip install optuna

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: C:\Users\shiva\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: C:\Users\shiva\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: C:\Users\shiva\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: C:\Users\shiva\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [2]:
%pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from pathlib import Path

import numpy as np
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import optuna
#import six
#sys.modules['sklearn.externals.six'] = six
#import joblib

#sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.classifier import EnsembleVoteClassifier
from mlxtend.feature_selection import ColumnSelector
#pd.set_option('display.max_columns', 38)

In [2]:
DATA_DIR = Path('data')
train_values = pd.read_csv(DATA_DIR / 'train_values.csv', index_col='building_id')
train_labels = pd.read_csv(DATA_DIR / 'train_labels.csv', index_col='building_id')

In [3]:
cat_cols=['geo_level_1_id','geo_level_2_id','geo_level_3_id','land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'legal_ownership_status','count_floors_pre_eq', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']

selected_features = ['foundation_type',
                     'land_surface_condition',
                     'roof_type',
                    'ground_floor_type',
                    'other_floor_type',
                    'position',
                    'plan_configuration',
                    'legal_ownership_status']

train_values_subset = train_values[selected_features]
train_values_subset = pd.get_dummies(train_values_subset)
merged_data = pd.concat([train_values, train_values_subset], axis='columns')

In [4]:
merged_data

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,1,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,1,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,1,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,1,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,1,0,0,0,0,1,0
669485,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,1,0
602512,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
print(format('How to find optimal parameters for CatBoost using GridSearchCV for Classification','*^82'))   
# Split the training data set
X = train_values; y = train_labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


#Initialise the catboost classifier, if you have no GPU on your machine you can remove task_type="GPU" 
model = CatBoostClassifier(eval_metric='TotalF1', task_type="GPU",cat_features=cat_cols)

#Choose parameters to test here
parameters = {'depth':[2,4,6,8,10,12],
            'iterations':[10,100,500,1000,5000],
            'learning_rate':[0.02,0.05,0.06,0.07], 
            'l2_leaf_reg':[3,5,7,9],
            'border_count':[11,13,15,17]}
print('Paramaters defined')

#Initialise the Gridsearch, cv is set to 2 for speed.
randm = GridSearchCV(estimator=model, param_grid = parameters,cv = 2, verbose = 2)
randm.fit(X_train, y_train, verbose = 0)

# Results from Random Search
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      randm.best_estimator_)

print("\n The best score across ALL searched params:\n",
      randm.best_score_)

print("\n The best parameters across ALL searched params:\n",
      randm.best_params_)

print("\n ========================================================")

How to find optimal parameters for CatBoost using GridSearchCV for Classification*
Paramaters defined
Fitting 2 folds for each of 1920 candidates, totalling 3840 fits
[CV] END border_count=11, depth=2, iterations=10, l2_leaf_reg=3, learning_rate=0.02; total time=   3.8s


KeyboardInterrupt: 

In [ ]:
print("\n The best estimator across ALL searched params:\n",
      randm.best_estimator_)

print("\n The best score across ALL searched params:\n",
      randm.best_score_)

print("\n The best parameters across ALL searched params:\n",
      randm.best_params_)

In [ ]:
print("hell")

In [ ]:
from sklearn.model_selection import cross_val_score
import optuna
def objective(trial):
    
        
    
    depth = trial.suggest_int('depth', 2,20)
    iterations = trial.suggest_int('iterations', 10,5000)
    learning_rate = trial.suggest_float('learning_rate', 0.02,0.3,step=0.01)
    l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 1,15)
    border_count = trial.suggest_int('border_count', 10,20)
    grow_policy = trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
    #eval_metric = trial.suggest_categorical('eval_metrics', ['TotalF1'])
    #xgbclassifier_colsample_bynode = trial.suggest_float('xgbclassifier_colsample_bynode', 0.5,1.0,step=0.1)

    estimator = CatBoostClassifier(eval_metric= 'TotalF1', task_type="GPU", grow_policy = grow_policy, cat_features=cat_cols, depth = depth, iterations = iterations,
                                  learning_rate = learning_rate, l2_leaf_reg = l2_leaf_reg, border_count = border_count, verbose = 0)

    #pipeline = make_pipeline(scaler,dimen_red_algorithm, estimator)
    
    score = cross_val_score(estimator, train_values, train_labels, scoring='f1_micro', cv=3)
    
    f1 = score.mean() # calculate the mean of scores
    return f1

search_space = {'depth':[2,4,6,8,10,12,14,16,18],
            'iterations':[10,100,500,1000,5000],
            'learning_rate':[0.02,0.05,0.06,0.07, 0.09,0.1, 0.15, 0.2], 
            'l2_leaf_reg':[1,3,5,7,9,11,13],
            'border_count':[11,13,15,17, 17],
            'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
            #'eval_metrics': ['TotalF1']
               }

sampler = optuna.samplers.GridSampler(search_space)

study = optuna.create_study(direction = 'maximize', sampler=sampler) # maximise the score during tuning

study.optimize(objective, n_trials=50) # run the objective function 100 times

print(study.best_trial) # print the best performing pipeline

[I 2023-04-20 11:42:15,152] A new study created in memory with name: no-name-5653254e-0c0c-448a-a5d7-2ada4455e42d
[I 2023-04-20 11:43:15,048] Trial 0 finished with value: 0.7406840342132227 and parameters: {'depth': 10, 'iterations': 1000, 'learning_rate': 0.06, 'l2_leaf_reg': 1, 'border_count': 13, 'grow_policy': 'Lossguide'}. Best is trial 0 with value: 0.7406840342132227.
[I 2023-04-20 11:43:28,955] Trial 1 finished with value: 0.7332128426214788 and parameters: {'depth': 8, 'iterations': 100, 'learning_rate': 0.06, 'l2_leaf_reg': 13, 'border_count': 15, 'grow_policy': 'Lossguide'}. Best is trial 0 with value: 0.7406840342132227.
[I 2023-04-20 11:43:38,711] Trial 2 finished with value: 0.72590281695005 and parameters: {'depth': 10, 'iterations': 10, 'learning_rate': 0.15, 'l2_leaf_reg': 1, 'border_count': 17, 'grow_policy': 'Lossguide'}. Best is trial 0 with value: 0.7406840342132227.
[I 2023-04-20 11:43:49,304] Trial 3 finished with value: 0.7045406579406833 and parameters: {'depth

In [ ]:
#{'depth': 10, 'iterations': 500, 'learning_rate': 0.06, 'l2_leaf_reg': 13, 'border_count': 13, 'grow_policy': 'Depthwise'}
#{'depth': 6, 'iterations': 5000, 'learning_rate': 0.05, 'l2_leaf_reg': 13, 'border_count': 17, 'grow_policy': 'Depthwise'}
model = CatBoostClassifier(eval_metric='TotalF1', task_type="GPU",cat_features=cat_cols, depth = 8, iterations = 5000,
                                  learning_rate = 0.05, l2_leaf_reg = 7, border_count = 11, verbose = 0)
model.fit(train_values,train_labels)

In [19]:
from sklearn.metrics import f1_score
#pred = pipe_2.predict(X_train)
#full_pred = model.predict(train_values)
#print(f1_score(train_labels, full_pred, average='micro'))

In [22]:
test_values = pd.read_csv(DATA_DIR / 'test_values.csv', index_col='building_id')

In [ ]:
predictions = model.predict(test_values)

In [ ]:
submission_format = pd.read_csv(DATA_DIR / 'submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)

my_submission['damage_grade'] =my_submission['damage_grade'].astype(int)
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [ ]:
my_submission.to_csv('catboost_test_2.csv')

## MLXTEND

In [34]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(merged_data.values)

ValueError: could not convert string to float: 't'

In [8]:
len(merged_data.columns[~merged_data.columns.isin(selected_features)].values)

68

In [51]:
from sklearn.pipeline import Pipeline
col_sel1 = ColumnSelector(cols=merged_data.columns[0:38].values)
col_sel2 = ColumnSelector(cols=merged_data.columns[~merged_data.columns.isin(selected_features)].values)

clf1 = xgb.XGBClassifier(tree_method='gpu_hist', n_estimators = 325, 
                                  max_depth = 24, max_leaves = 0, 
                                  eta = 0.060000000000000005,random_state=42,
                                  subsample = 0.8, colsample_bytree = 0.5, colsample_bylevel = 1.0,
                                  min_child_weight = 12, gamma = 0.5, 
                                  objective = "multi:softmax", num_class=3, reg_lambda = 1.4200000000000002, reg_alpha = 0.7000000000000001)





clf2 = lgb.LGBMClassifier(n_estimators = 325, 
                                  max_depth = 24, num_leaves = 131072, 
                                  learning_rate = 0.060000000000000005,random_state=42,
                                  subsample = 0.8, colsample_bytree = 0.5, colsample_bylevel = 1.0,
                                  min_child_weight = 12, min_gain_to_split = 0.4,
                                  objective = "multiclass", num_class=3, reg_lambda = 1.4200000000000002, reg_alpha = 0.7000000000000001)

# clf3 = CatBoostClassifier(eval_metric='TotalF1', task_type="GPU",cat_features=cat_cols, depth = 8, iterations = 5000,
#                                    learning_rate = 0.05, l2_leaf_reg = 7, border_count = 11, verbose = 0)

clf3 = CatBoostClassifier(eval_metric='TotalF1', task_type="GPU",cat_features=cat_cols, depth = 16, iterations = 500, random_state = 42,
                                   learning_rate = 0.07, l2_leaf_reg = 3, border_count = 17, grow_policy = 'Depthwise', verbose = 0)

clf1_pipe = Pipeline([('sel', col_sel2),
                      ('logreg', clf1)])

clf2_pipe = Pipeline([('sel', col_sel2),
                      ('logreg', clf2)])

clf3_pipe = Pipeline([('sel', col_sel1),
                      ('logreg', clf3)])

In [10]:
(y_train.values.ravel()-1).dtype

NameError: ignored

In [5]:

X_train, X_test, y_train, y_test = train_test_split(merged_data, train_labels, test_size=0.2, shuffle = True, random_state=42)

,damage_grade
building_id,
771955,2
162193,2
336662,2
904537,1
117658,2
...,...
876414,3
939227,1
296737,2


In [13]:
eclf = EnsembleVoteClassifier(clfs=[clf1_pipe, clf2_pipe, clf3_pipe], 
                              voting='soft')

In [14]:
eclf.fit(X_train,y_train.values.ravel()-1)

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] min_gain_to_split is set=0.4, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.4


CatBoostError: ignored

In [15]:
test = col_sel1.transform(X_train)
test

array([[13, 257, 8352, ..., 0, 0, 0],
       [6, 1076, 9202, ..., 0, 0, 0],
       [7, 838, 10723, ..., 0, 0, 0],
       ...,
       [8, 1073, 5249, ..., 0, 0, 0],
       [10, 76, 1841, ..., 0, 0, 0],
       [7, 746, 10603, ..., 0, 0, 0]], dtype=object)

In [6]:
from sklearn.ensemble import VotingClassifier
#merged_data.drop(selected_features, axis='columns', inplace = True)

In [7]:
X_train_cat = X_train[train_values.columns.values]
X_train_other = X_train.drop(selected_features, axis='columns')

In [8]:
X_test_cat = X_test[train_values.columns.values]
X_test_other = X_test.drop(selected_features, axis='columns')

In [9]:
X_train_cat

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
739427,13,257,8352,3,25,2,5,n,r,n,...,0,0,0,0,0,0,0,0,0,0
17201,6,1076,9202,3,40,10,8,o,r,n,...,0,0,1,0,0,0,0,0,0,0
723805,7,838,10723,2,45,8,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
891512,7,555,2763,2,30,7,4,t,r,n,...,0,0,0,0,0,0,0,0,0,0
484350,17,682,1039,3,30,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79888,13,909,1626,2,25,6,5,t,r,q,...,0,0,0,0,0,0,0,0,0,0
442764,10,1397,8182,2,5,5,4,n,r,n,...,0,0,0,0,0,0,0,0,0,0
17256,8,1073,5249,3,20,22,7,o,r,q,...,0,1,0,0,0,0,0,0,0,0


In [53]:
# eclf = VotingClassifier(
#      estimators=[('xgb', clf1), ('lgbm', clf2), ('cat', clf3)],
#     voting='soft')

classifiers = [('xgb', clf1), ('lgbm', clf2), ('cat', clf3)]

In [54]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

def fit_multiple_estimators(classifiers, X_list, y, sample_weights = None):

    # Convert the labels `y` using LabelEncoder, because the predict method is using index-based pointers
    # which will be converted back to original data later.
    le_ = LabelEncoder()
    le_.fit(y)
    transformed_y = le_.transform(y)

    # Fit all estimators with their respective feature arrays
    estimators_ = [clf.fit(X, y) if sample_weights is None else clf.fit(X, y, sample_weights) for clf, X in zip([clf for _, clf in classifiers], X_list)]

    return estimators_, le_


def predict_from_multiple_estimator(estimators, label_encoder, X_list, weights = None):

    # Predict 'soft' voting with probabilities

    pred1 = np.asarray([clf.predict_proba(X) for clf, X in zip(estimators, X_list)])
    pred2 = np.average(pred1, axis=0, weights=weights)
    pred = np.argmax(pred2, axis=1)

    # Convert integer predictions to original labels:
    return label_encoder.inverse_transform(pred)

In [55]:
X_train_list = [X_train_other, X_train_other, X_train_cat]
fitted_estimators, label_encoder = fit_multiple_estimators(classifiers, X_train_list, y_train.values.ravel()-1)

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] min_gain_to_split is set=0.4, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.4


In [14]:
X_test_list = [X_test_other, X_test_other, X_test_cat]

In [63]:
#1.06682501, 0.8434502 , 1.04286667 1.1609375 , 0.93671875, 1.0
#y_pred = predict_from_multiple_estimator(fitted_estimators, label_encoder, X_test_list, [1.06682501, 0.8434502 , 1.04286667])
#1.01601562, 1.04140625, 1.0
y_pred = predict_from_multiple_estimator(fitted_estimators, label_encoder, X_test_list, [0.95991211, 0.99814453, 1.0])


In [64]:
#train_pred = predict_from_multiple_estimator(fitted_estimators, label_encoder, X_train_list, [1.06682501, 0.8434502 , 1.04286667])
train_pred = predict_from_multiple_estimator(fitted_estimators, label_encoder, X_train_list, [0.95991211, 0.99814453, 1.0])

In [65]:
print(f1_score(y_test.values.ravel(), y_pred +1, average='micro'))

0.7500047965311487


In [66]:
print(f1_score(y_train.values.ravel(), train_pred +1, average='micro'))

0.8324539524174981


In [44]:
cat_data_cols

array(['count_floors_pre_eq', 'age', 'area_percentage',
       'height_percentage', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick',
       'has_superstructure_timber', 'has_superstructure_bamboo',
       'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other'],
      dtype=object)

In [43]:
train_values.dtypes

geo_level_1_id                              int64
geo_level_2_id                              int64
geo_level_3_id                              int64
count_floors_pre_eq                       float64
age                                       float64
area_percentage                           float64
height_percentage                         float64
land_surface_condition                     object
foundation_type                            object
roof_type                                  object
ground_floor_type                          object
other_floor_type                           object
position                                   object
plan_configuration                         object
has_superstructure_adobe_mud              float64
has_superstructure_mud_mortar_stone       float64
has_superstructure_stone_flag             float64
has_superstructure_cement_mortar_stone    float64
has_superstructure_mud_mortar_brick       float64
has_superstructure_cement_mortar_brick    float64


In [34]:
final_other = merged_data.drop(selected_features, axis='columns')
# scaler1 = MinMaxScaler()
# scaler2 = MinMaxScaler()

# cat_data_cols = train_values.columns[~train_values.columns.isin(selected_features + ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'])].values
# other_data_cols = final_other.columns[~final_other.columns.isin(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'])]
# train_values.loc[:, cat_data_cols] = scaler1.fit_transform(train_values[cat_data_cols])
# final_other.loc[:, other_data_cols] = scaler2.fit_transform(final_other[other_data_cols])

In [67]:
clf1 = xgb.XGBClassifier(tree_method='gpu_hist', n_estimators = 325, 
                                  max_depth = 24, max_leaves = 0, 
                                  eta = 0.060000000000000005,random_state=42,
                                  subsample = 0.8, colsample_bytree = 0.5, colsample_bylevel = 1.0,
                                  min_child_weight = 12, gamma = 0.5, 
                                  objective = "multi:softmax", num_class=3, reg_lambda = 1.4200000000000002, reg_alpha = 0.7000000000000001)





clf2 = lgb.LGBMClassifier(n_estimators = 325, 
                                  max_depth = 24, num_leaves = 131072, 
                                  learning_rate = 0.060000000000000005,random_state=42,
                                  subsample = 0.8, colsample_bytree = 0.5, colsample_bylevel = 1.0,
                                  min_child_weight = 12, min_gain_to_split = 0.4,
                                  objective = "multiclass", num_class=3, reg_lambda = 1.4200000000000002, reg_alpha = 0.7000000000000001)

clf3 = CatBoostClassifier(eval_metric='TotalF1', task_type="GPU",cat_features=cat_cols, depth = 16, iterations = 500, random_state = 42,
                                   learning_rate = 0.07, l2_leaf_reg = 3, border_count = 17, grow_policy = 'Depthwise', verbose = 0)

classifiers = [('xgb', clf1), ('lgbm', clf2), ('cat', clf3)]

fitted_estimators, label_encoder = fit_multiple_estimators(classifiers, [final_other, final_other, train_values], train_labels.values.ravel()-1)

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] min_gain_to_split is set=0.4, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.4


In [36]:
test_values = pd.read_csv(DATA_DIR / 'test_values.csv', index_col='building_id')

In [37]:
test_values_subset = test_values[selected_features]
test_values_subset = pd.get_dummies(test_values_subset)

In [38]:
merged_data_test = pd.concat([test_values, test_values_subset], axis='columns')
merged_data_test.drop(selected_features, axis='columns', inplace = True)

In [51]:
#merged_data_test.drop(selected_features, axis='columns', inplace = True)
merged_data_test.loc[:, other_data_cols] = scaler2.transform(merged_data_test[other_data_cols])

In [39]:
merged_data_test

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,0,1,0,...,0,0,0,0,0,0,0,0,1,0
99355,6,141,11987,2,25,13,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
890251,22,19,10044,2,5,4,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
745817,26,39,633,1,0,19,3,0,0,0,...,0,0,0,0,0,0,0,0,1,0
421793,17,289,7970,3,15,8,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,0,1,0,...,0,0,0,0,0,0,0,0,0,1
663567,10,1407,11907,3,25,6,7,1,1,1,...,0,0,0,0,0,0,0,0,1,0
1049160,22,1136,7712,1,50,3,3,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [68]:
predictions = predict_from_multiple_estimator(fitted_estimators, label_encoder, [merged_data_test, merged_data_test, test_values], [0.95991211, 0.99814453, 1.0])

In [69]:
submission_format = pd.read_csv(DATA_DIR / 'submission_format.csv', index_col='building_id')

In [70]:
my_submission = pd.DataFrame(data=predictions+1,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [76]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [77]:
my_submission.to_csv('ensemble_test_0.95991211_0.99814453_1.0.csv')

In [22]:
class TestEnsemble:
    def __init__(self, weights, estimator = None, label_encoder = None):
        self.weights = weights
        self.clf1 = xgb.XGBClassifier(tree_method='gpu_hist', n_estimators = 325, 
                                  max_depth = 24, max_leaves = 0, 
                                  eta = 0.060000000000000005,random_state=42,
                                  subsample = 0.8, colsample_bytree = 0.5, colsample_bylevel = 1.0,
                                  min_child_weight = 12, gamma = 0.5, 
                                  objective = "multi:softmax", num_class=3, reg_lambda = 1.4200000000000002, reg_alpha = 0.7000000000000001)





        self.clf2 = lgb.LGBMClassifier(n_estimators = 325, 
                                          max_depth = 24, num_leaves = 131072, 
                                          learning_rate = 0.060000000000000005,random_state=42,
                                          subsample = 0.8, colsample_bytree = 0.5, colsample_bylevel = 1.0,
                                          min_child_weight = 12, min_gain_to_split = 0.4,
                                          objective = "multiclass", num_class=3, reg_lambda = 1.4200000000000002, reg_alpha = 0.7000000000000001)

        self.clf3 = CatBoostClassifier(eval_metric='TotalF1', task_type="GPU",cat_features=cat_cols, depth = 8, iterations = 5000,
                                           learning_rate = 0.05, l2_leaf_reg = 7, border_count = 11, verbose = 0)

        self.classifiers = [('xgb', self.clf1), ('lgbm', self.clf2), ('cat', self.clf3)]
        self.estimators = estimator
        self.label_encoder = label_encoder
        
    def fit_multiple_estimators(self,classifiers, X_list, y, sample_weights = None):

        # Convert the labels `y` using LabelEncoder, because the predict method is using index-based pointers
        # which will be converted back to original data later.
        le_ = LabelEncoder()
        le_.fit(y)
        transformed_y = le_.transform(y)

        # Fit all estimators with their respective feature arrays
        estimators_ = [clf.fit(X, y) if sample_weights is None else clf.fit(X, y, sample_weights) for clf, X in zip([clf for _, clf in classifiers], X_list)]

        return estimators_, le_


    def predict_from_multiple_estimator(self,estimators, label_encoder, X_list, weights = None):

        # Predict 'soft' voting with probabilities

        pred1 = np.asarray([clf.predict_proba(X) for clf, X in zip(estimators, X_list)])
        pred2 = np.average(pred1, axis=0, weights=weights)
        pred = np.argmax(pred2, axis=1)

        # Convert integer predictions to original labels:
        return label_encoder.inverse_transform(pred)
    
    def fit(self, X,y):
        self.estimators, self.label_encoder = fit_multiple_estimators(self.classifiers, X, y, sample_weights = None)
        
    def predict(self,X):
        predictions = self.predict_from_multiple_estimator(self.estimators, self.label_encoder, X, self.weights)
        return predictions
        
    def score(self,X,y):
        predictions = self.predict(X)
        return f1_score(y.values.ravel(), predictions +1, average='micro')

        

In [60]:
from scipy.optimize import minimize


def function_to_minimize(weights):

    w1, w2 = weights  # these are the new weights!

    #Use Classifier fitted on split data not total
    newclf = TestEnsemble([w1,w2,1.0], fitted_estimators,label_encoder)

#     newclf.fit(new_data, train_labels.values.ravel()-1)
#     score = newclf.score(new_data, train_labels.values.ravel()-1)
    #score = cross_val_score(newclf, new_data, train_labels.values.ravel()-1, scoring='f1_micro').mean()
    
    #newclf.fit(X_train_list, y_train.values.ravel()-1)
    score = newclf.score(X_test_list, y_test)
    
    # change accuracy to error so that smaller is better
    score_to_minimize = 1 - score

    return score_to_minimize

In [61]:
init_weights = [1., 1.]

results = minimize(
    function_to_minimize,
    init_weights,
    bounds=((0, 5), (0, 5)),
    method="nelder-mead",
)

In [62]:
results

 final_simplex: (array([[0.95991211, 0.99814453],
       [0.95993729, 0.99815521],
       [0.95984097, 0.9981617 ]]), array([0.2499952, 0.2499952, 0.2499952]))
           fun: 0.2499952034688513
       message: 'Optimization terminated successfully.'
          nfev: 49
           nit: 17
        status: 0
       success: True
             x: array([0.95991211, 0.99814453])

In [ ]:
#1.1609375 , 0.93671875
#1.00208333, 1.04965278, 0.99490741